# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
This is a classification problem as:
- it requires the output to be in only one of two distinct categories: 'yes' - early intervention is required; or 'no' early intervention required.
- the output variable does <b>not</b> take on continuous values (which would be a feature of regression problems)

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [6]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [7]:
# Calculate number of students
n_students = student_data.shape[0]

# Calculate number of features
# n_features = len(student_data.columns[:-1]) # excluding last column.  Alternative way of deriving this
n_features = student_data.shape[1]-1 # excluding the target column

# Calculate passing students
n_passed = len(student_data[student_data['passed']=="yes"])

# Calculate failing students
n_failed = len(student_data[student_data['passed']=="no"])

# Calculate graduation rate
grad_rate = float(n_passed)/n_students*100


# Print the results
print "Total number of students: {}".format(n_students)

print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [8]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [9]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [10]:
# Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# Shuffle and split the dataset into the number of training and testing points above
X_train,X_test,y_train,y_test=train_test_split(X_all,y_all,test_size=num_test,random_state=0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


C:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
3 supervised learning models that would be appropriate and their supporting details are listed below:

### 1. Decision Trees

** Example application:** Decision trees have been used to filter noise from Hubble Space Telescope images.

**Advantages:** 
* easy to interpret and explain.  
* It takes into account feature interactions and because it is non parametric, data outliers and the linear separability of the data are less of a concern than with other methods (i.e. splitting happens based on the proportion of samples within the split ranges rather than on absolute values).
* Does not require as much data preparation.  For example, a regression model requires data variables to be normalized or scales in order to better fit the model and interpret the model coefficients.  
* A decision tree is also not impacted by missing values.
* Does not require any assumption of linearity in data and is not impacted by non linear relationships between variables.

**Disadvantages:** 
- Can easily overfit to data (although this can be addressed by ensemble methods like random forests).
- Learning algorithm may be non-optimal and suffer from local minima.

**Appropriateness for problem:** 
- Decision trees implicitly perform feature selection and provides and indication of the importance of each feature.  For example, the top few nodes of a tree are the most important within the dataset.  This is desirable for this problem as it not only allows for the identification of students for intervention, it also provides insights into how to intervene.

### 2. SVM
** Example application:** SVMs have been used in handwriting character recognition problems. It is especially popular in text recognition problems where very high dimensional spaces are the norm.

** Advantages:**
- good for large feature sets as it is able to separate classes more quickly and with less overfitting than many other algorithms.  It is able to cope with large numbers of features as it does not operate on the source feature space explicitly.  Instead, it only calculates the 'distances' between each data point, which remains invariant with the number of features.  Hence, because instances from the training set only appear as arguments of the kernel function and even though usage of the kernels often then increase feature dimensionality (virtually) up to the infinite dimensional space, the number of 'distances' between data points does not change.  *The trade off, however, is that the number of distances increases dramatically as the number of points increases!* 
- Has theoretical guarantees with regards to overfitting, and thanks to the kernel trick, it can be used even on data that is not linearly separable in the base feature space. 
- Kernel trick also allows for the incorporation of expert domain knowledge into solving the problem via the engineering of the kernel.
- Resulting learning algorithm is an optimized solution and avoids the 'local minimum' problem of methods like neural nets and decision trees.

** Disadvantages:** 
- hard to interpret and difficult to take into account new data.  Similar to other linear algorithms, it assumes linear class boundaries i.e. assumes that classes can be separated by a straight line (or its higher dimension analog).  This implies that data trends follow a straight line, an assumption which may not be valid for some problems.  
- Not efficient if the number of features is very large in comparison to the training samples.
- Not especially good for multi class classification as in this instance, it is effectively a binary classifier with some kind of voting mechanism.

** Appropriateness for problem:** 
- The SVM's ability to handle large number of features (31 in this example) and the fact that this is just a binary classification problem makes this a probable learning model.

### 3. Logistic Regression

** Example application:** logistic regression has been used in healthcare to analyse over a million people for myocardial infarction over a 10 year period.

** Advantages:** 
- fast and simple.  
- results have a probabilistic interpretation (unlike SVMs or decision trees) and so this method is especially useful for problems with a probabilistic framework.  
- models can be easily updated to take into account new data using a gradient descent method.  Hence, the model is useful in instances when more training data is expected in the future.

** Disadvantages:** 
- Assumes linear class boundaries i.e. assumes that classes can be separated by a straight line (or its higher dimension analog).  This implies that data trends follow a straight line, an assumption which may not be valid for some problems.  However, despite this, linear algorithms like this tend to be a very popular 'first line of attack' as they tend to be algorithmically simpler and faster to train.

** Appropriateness for problem:** 
- As the problem is a binary classification problem, and the result is probabilistic, this method can help to identify those students ***most*** in need of intervention.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_classifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [11]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [13]:
# Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC

# Initialize the three models
clf_A = DecisionTreeClassifier(random_state=0)
clf_B = LogisticRegression(random_state=0)
clf_C = SVC(random_state=0)

# Set up the training set sizes
#test_size_100=X_train.shape[0]-100
#test_size_200=X_train.shape[0]-200
#test_size_300=X_train.shape[0]-300

#X_train_100,X_test_100,y_train_100,y_test_100=train_test_split(X_train,y_train,test_size=test_size_100,random_state=0)
#X_train_200,X_test_200,y_train_200,y_test_200=train_test_split(X_train,y_train,test_size=test_size_200,random_state=0)
#X_train_300,X_test_300,y_train_300,y_test_300=train_test_split(X_train,y_train,test_size=test_size_300,random_state=0)


In [14]:
# Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
#A_100=train_predict(clf_A,X_train_100,y_train_100,X_test,y_test)
#B_100=train_predict(clf_B,X_train_100,y_train_100,X_test,y_test)
#C_100=train_predict(clf_C,X_train_100,y_train_100,X_test,y_test)

for clf in [clf_A,clf_B,clf_C]:
    # show estimator type
    print "\n{}: \n".format(clf.__class__.__name__)
    
    # looping through training sizes
    for n in [100,200,300]:
        train_predict(clf,X_train[:n],y_train[:n],X_test,y_test)


DecisionTreeClassifier: 

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6942.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7132.
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0020 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7167.

LogisticRegression: 

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0140 seconds
Made predictions in 0.0360 seconds.
F1 score for training set: 0.8571.
Made predictions in 0.0010 seconds.
F1 score for test

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Decision Tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0010           | 0.0010                 |        1.00      |      0.6942     |
| 200               |        0.0010           | 0.0000                 | 1.00             |      0.7132     |
| 300               |        0.0020           | 0.0010                 | 1.00             |     0.7167      |

** Classifer 2 - Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0140              | 0.0010                 | 0.8571           |    0.7612       |
| 200               |     0.0020              | 0.0000                 | 0.8380           |    0.7794       |
| 300               |     0.0040              | 0.0000                 | 0.8381           |    0.7910       |

** Classifer 3 - Support Vector Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0010              | 0.0010                 | 0.8591           |     0.7838      |
| 200               |     0.0040              | 0.0020                 | 0.8693           |     0.7755      |
| 300               |     0.0080              | 0.0020                 | 0.8692           |     0.7586      |

**QUERY:** I don't understand the performance times on the logistic regression model, in particular why it takes so much longer to train and predict a model based on just 100 data points.  I tried this several times and on different machines and have observed the same result.  Is this as expected?

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Dear Board of Supervisors,

I write to recommend using a logistic regression model.  I make my recommendation on the basis of the following factors:

#### Accuracy
- the F1 score is the metric used to measure the accuracy of the model's predictions.  The F1 value reaches its best value at 1, and its worst at 0.  
- I have focused my attention on the predictive accuracy of each model on the test set, rather than the training set accuracy as the former is more indicative of the success of the models going forward.
- The best test accuracy is observed with SVMs and logistic regression ("LR") models.  (In fact, the worst accuracy performance from both the SVM and LR models over all training set sizes is still better than the best performance from the decision tree model.)  
- The LR model shows an improvement in its test set predictions as the number of training data points increase.  This is not what has been observed in the SVM (which has the opposite effect i.e. performance worsening as more training data points are introduced) and the decision tree models.  I therefore expect that as more data becomes available in the future, we should witness more improvements in the performance of the LR model.

#### Computational Costs
##### i. Training time:
- the model with the best overall training time performance is the decision tree, followed by logistic regression and SVM.
- As the number of training data points increase, the SVM model performance becomes significantly worse (i.e. deteriorates to less than half the speed of the other two models).

#### ii. Prediction time:
- the models with the best prediction times are the decision tree and LR models.  
- in comparison to the other two models, the SVM is very significantly slower in predicting outcomes. 


In summary, the decision tree model has the best computational performance but the worst accuracy.  Of the two remaining models, the SVM has poorer computational performance.  Hence, my recommendation is to go with the logistic regression model.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Regression analysis is aimed at allowing us to make predictions of an outcome based on some inputs.  In this instance, the outcome we want to predict is whether a student will fail to graduate.  The inputs we have provided is a range of socio-economic factors such as the student's family background and parents' educational levels.

Linear regression is a version of regression analysis which multiplies each input by a coefficient, sums the values and adds a constant.  The output is then simply the result of this calculation.  The training process involves determining the values of the constant and coefficients for each input that best reflect the training data. 

Logistic regression is a regression method that builds on linear regression by applying a data transformation to force the outputs from the linear regression calculation to take on values between 0 and 1 (the output from linear regression is effectively unbounded i.e. it can be a very large or very small number as well as take on any values in between).  These values can then be interpreted as 'probabilities'.  In this example, the output can be regarded as the 'probability' that a student will fail to graduate.  Instances where the probabilities are >0.5 are then classed as 'yes' and those <=0.5 are classed as 'no'.  The training of a logistic regression model is similar to that for a linear regression model - the constant value and the coefficients of inputs are determined by maximising the accuracy of resulting predictions on the training data.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

The following are the parameters of the logistic regression model:

In [17]:
clf_B.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [18]:
# the purpose of this code chunk is to see the distribution of labels in the test set 
import pandas
#ps=pandas.Series([tuple(i) for i in y_test])
ps=pandas.Series([(i) for i in y_test])
counts=ps.value_counts()
print counts

yes    59
no     36
dtype: int64


In [35]:
# Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
#from sklearn.model_selection import GridSearchCV
#from sklearn.cross_validation import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score # for scoring function

# Create the parameters list you wish to tune
#parameters = [{'penalty':['l1','l2'],'C':[0.1,0.5,1.0]}]
parameters = [{'penalty':['l1','l2'],'C':list(np.arange(0.1,1.1,0.1))}]

# Set classifier to current settings for the chosen classifier
clf = clf_B 

# Make an f1 scoring function using 'make_scorer' 
# Note to self: the F1 score can be interpreted as a weighted average of the precision and recall.  The F1 score reaches
# its best value at 1 i.e. the higher the F1 score, the better.
# the 'greater_is_better' parameter is set to True even though its default value is True to 
# make explicit that the higher the F1 score, the better.

#f1_scorer = make_scorer(f1_score,average="binary",pos_label='yes',greater_is_better=True)
f1_scorer = make_scorer(f1_score,pos_label='yes',greater_is_better=True)

# Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid=parameters,scoring=f1_scorer)


# Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0000 seconds.
Tuned model has a training F1 score of 0.8308.
Made predictions in 0.0000 seconds.
Tuned model has a testing F1 score of 0.7943.


#### View grid scores

In [36]:
from IPython.display import display
display(pd.DataFrame(grid_obj.grid_scores_))

,parameters,mean_validation_score,cv_validation_scores
0,"{u'penalty': u'l1', u'C': 0.1}",0.825445,"[0.824242424242, 0.819875776398, 0.832298136646]"
1,"{u'penalty': u'l2', u'C': 0.1}",0.810225,"[0.797385620915, 0.818181818182, 0.815286624204]"
2,"{u'penalty': u'l1', u'C': 0.2}",0.825944,"[0.827160493827, 0.820512820513, 0.830188679245]"
3,"{u'penalty': u'l2', u'C': 0.2}",0.798538,"[0.761904761905, 0.828947368421, 0.805194805195]"
4,"{u'penalty': u'l1', u'C': 0.3}",0.818788,"[0.820512820513, 0.820512820513, 0.815286624204]"
5,"{u'penalty': u'l2', u'C': 0.3}",0.795683,"[0.753424657534, 0.828947368421, 0.805194805195]"
6,"{u'penalty': u'l1', u'C': 0.4}",0.815396,"[0.805194805195, 0.825806451613, 0.815286624204]"
7,"{u'penalty': u'l2', u'C': 0.4}",0.787601,"[0.744827586207, 0.828947368421, 0.789473684211]"
8,"{u'penalty': u'l1', u'C': 0.5}",0.812008,"[0.797385620915, 0.823529411765, 0.815286624204]"
9,"{u'penalty': u'l2', u'C': 0.5}",0.781402,"[0.744827586207, 0.82119205298, 0.778523489933]"


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

As per the outputs above and below, when comparing the tuned versus untuned model's F1, the results are as follows:
- test data F1: tuned model is higher (0.7943 vs 0.7910) (*vs 0.8000 for tuned model on normalized data, see below*)
- training data F1: tuned model is lower (0.8308 vs 0.8381) (*vs 0.8351 for tuned model on normalized data, see below*)

** Query: ** 
I don't understand the training F1 results.  Isn't the GridSearchCV object/function supposed to find the optimal model by fine tuning the model parameters?  Since the search parameter list contains the default parameters from the initial classifier and is using the same training data and measure of performance, I don't see how this result is possible?  

** Edit: ** 
It is possible that grid search will choose a set of parameters that performs worse than just the default settings because it uses a cross validation approach in choosing the optimal parameter setting (This is where the training data is further subdivided into sub-samples to allow for training-testing cycles to be performed on each sub-sample and  the *average* performance across these sub samples is used as the performance measure, rather than the result from a *single* test set).

In [38]:
clf.get_params()

{'C': 0.20000000000000001,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l1',
 'random_state': 0,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

#### Additional Model Investigations

In [43]:
from sklearn.preprocessing import normalize
X_train_norm=normalize(X_train)
X_test_norm=normalize(X_test)

clf_B_norm=clf_B
result_norm=train_predict(clf_B_norm,X_train_norm,y_train,X_test_norm,y_test)

Training a LogisticRegression using a training set size of 300. . .
Trained model in 0.0020 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8142.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7662.


In [44]:
clf_B_norm.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [55]:
params_norm={
    'C':np.logspace(-4,4,17),
    'penalty':['l1','l2'],
    'class_weight':[None,'balanced']
    }

grid_obj_norm = GridSearchCV(clf_B_norm, param_grid=params_norm,scoring=f1_scorer)


# Fit the grid search object to the training data and find the optimal parameters
grid_obj_norm.fit(X_train_norm,y_train)

# Get the estimator
clf_norm = grid_obj_norm.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model with normalized parameters has a training F1 score of {:.4f}.".format(predict_labels(clf_norm, X_train_norm, y_train))
print "Tuned model with normalized parameters has a testing F1 score of {:.4f}.".format(predict_labels(clf_norm, X_test_norm, y_test))


C:\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Made predictions in 0.0010 seconds.
Tuned model with normalized parameters has a training F1 score of 0.8351.
Made predictions in 0.0000 seconds.
Tuned model with normalized parameters has a testing F1 score of 0.8000.


In [56]:
clf_B_norm.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [57]:
display(pd.DataFrame(grid_obj_norm.grid_scores_))

,parameters,mean_validation_score,cv_validation_scores
0,"{u'penalty': u'l1', u'C': 0.0001, u'class_weig...",0.000000,"[0.0, 0.0, 0.0]"
1,"{u'penalty': u'l2', u'C': 0.0001, u'class_weig...",0.814226,"[0.811764705882, 0.816568047337, 0.814371257485]"
2,"{u'penalty': u'l1', u'C': 0.0001, u'class_weig...",0.000000,"[0.0, 0.0, 0.0]"
3,"{u'penalty': u'l2', u'C': 0.0001, u'class_weig...",0.777416,"[0.751677852349, 0.807453416149, 0.773333333333]"
4,"{u'penalty': u'l1', u'C': 0.000316227766017, u...",0.000000,"[0.0, 0.0, 0.0]"
5,"{u'penalty': u'l2', u'C': 0.000316227766017, u...",0.814226,"[0.811764705882, 0.816568047337, 0.814371257485]"
6,"{u'penalty': u'l1', u'C': 0.000316227766017, u...",0.000000,"[0.0, 0.0, 0.0]"
7,"{u'penalty': u'l2', u'C': 0.000316227766017, u...",0.774932,"[0.751677852349, 0.8, 0.773333333333]"
8,"{u'penalty': u'l1', u'C': 0.001, u'class_weigh...",0.000000,"[0.0, 0.0, 0.0]"
9,"{u'penalty': u'l2', u'C': 0.001, u'class_weigh...",0.814226,"[0.811764705882, 0.816568047337, 0.814371257485]"


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.